In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd 
%cd drive/MyDrive/models/SPQ/

/content
/content/drive/MyDrive/models/SPQ


In [3]:
!pip install kornia

     |████████████████████████████████| 474 kB 5.0 MB/s 


In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
!nvidia-smi

Sun Mar  6 07:47:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    35W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from utils import *
from tqdm import tqdm
from dataset import SingleData
import math

def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    half=math.floor(len(eval_img_list)/2)
    print(half)
    eval_=eval_img_list[:half]
    test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list

def pqDist_one(C, N_books, g_x, q_x,query_pname,gallery_pname):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    D_C = T.zeros((l1, N_books), dtype=T.float32)

    q_x_split = T.split(q_x, L_word, 0)
    g_x_split = np.split(g_x.cpu().data.numpy(), N_books, 1)
    C_split = T.split(C, L_word, 1)
    D_C_split = T.split(D_C, 1, 1)

    for j in range(N_books):
        for k in range(l1):
            # D_C_split[j][k] =T.norm(q_x_split[j]-C_split[j][k], 2)
            D_C_split[j][k] = T.norm(q_x_split[j]-C_split[j][k], 2).detach() #for PyTorch version over 1.9
        if j == 0:
            dist = D_C_split[j][g_x_split[j]]
        else:
            dist = T.add(dist, D_C_split[j][g_x_split[j]])
    Dpq = T.squeeze(dist)
#     ##### TASK
#     for i in range(len(gallery_pname)):
#         if query_pname==gallery_pname[i]:
#             Dpq[i]=float('inf')
#     #####
    return Dpq

def Indexing(C, N_books, X):
    l1, l2 = C.shape
    L_word = int(l2/N_books)
    x = T.split(X, L_word, 1)
    y = T.split(C, L_word, 1)
    for i in range(N_books):
        diff = squared_distances(x[i], y[i])
        arg = T.argmin(diff, dim=1)
        min_idx = T.reshape(arg, [-1, 1])
        if i == 0:
            quant_idx = min_idx
        else:
            quant_idx = T.cat((quant_idx, min_idx), dim=1)
    return quant_idx

def Evaluate_mAP(C, N_books, gallery_codes, query_codes, gallery_labels, query_labels, device,gallery_pname,query_pname, TOP_K=None):
    num_query = query_labels.shape[0]
    mean_AP = 0.0

    with tqdm(total=num_query, desc="Evaluate mAP", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
        for i in range(num_query):
            # Retrieve images from database
            retrieval = (query_labels[i, :] @ gallery_labels.t() > 0).float()

            # Arrange position according to hamming distance
            retrieval = retrieval[T.argsort(pqDist_one(C, N_books, gallery_codes, query_codes[i],query_pname[i],gallery_pname))][:TOP_K]

            # Retrieval count
            retrieval_cnt = retrieval.sum().int().item()

            # Can not retrieve images
            if retrieval_cnt == 0:
                continue

            # Generate score for every position
            score = T.linspace(1, retrieval_cnt, retrieval_cnt).to(device)

            # Acquire index
            index = (T.nonzero(retrieval == 1, as_tuple=False).squeeze() + 1.0).float().to(device)

            mean_AP += (score / index).mean()
            pbar.update(1)

        mean_AP = mean_AP / num_query
    return mean_AP

def DoRetrieval(device, args, net, C):
    print("Do Retrieval!")


    class_name, train_img_list = get_data_list('content/drive/MyDrive/dataSets/CRC/train')
    train_data_set=SingleData(class_name, train_img_list, transform=transforms.ToTensor())
    Gallery_loader = T.utils.data.DataLoader(train_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    # trainset = torchvision.datasets.CIFAR10(root=args.data_dir, train=True, download=args.if_download, transform=transforms.ToTensor())
    # Gallery_loader = T.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)
    class_test_name, test_img_list = get_data_list('content/drive/MyDrive/dataSets/CRC/same test')
    test_data_set = SingleData(class_test_name, test_img_list, transform=transforms.ToTensor())    
    
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    Query_loader = T.utils.data.DataLoader(test_data_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    net.eval()
    with T.no_grad():
        with tqdm(total=len(Gallery_loader), desc="Build Gallery", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
            for i, data in enumerate(Gallery_loader, 0):
                gallery_x_batch, gallery_y_batch = data[0].to(device), data[1].to(device)
                gallery_y_batch=gallery_y_batch.long()
                outputs = net(gallery_x_batch)
                gallery_c_batch = Indexing(C, args.N_books, outputs[0])
                gallery_y_batch = T.eye(args.num_cls)[gallery_y_batch]
                temp=[]
                for s in data[2]:
                    index1=s.rfind('/')
                    temp.append(s[index1+4:index1+7])
                    print(s[index1+4:index1+7])
                    
                if i == 0:
                    gallery_c = gallery_c_batch
                    gallery_y = gallery_y_batch
                    gallery_pname=temp
                else:
                    gallery_c = T.cat([gallery_c, gallery_c_batch], 0)
                    gallery_y = T.cat([gallery_y, gallery_y_batch], 0)
                    gallery_pname=gallery_pname + temp
                pbar.update(1)

        with tqdm(total=len(Query_loader), desc="Compute Query", bar_format='{desc:<15}{percentage:3.0f}%|{bar:10}{r_bar}') as pbar:
            for i, data in enumerate(Query_loader, 0):
                query_x_batch, query_y_batch = data[0].to(device), data[1].to(device)
                query_y_batch=query_y_batch.long()
                outputs = net(query_x_batch)
                query_y_batch = T.eye(args.num_cls)[query_y_batch]
                
                temp=[]
                for s in data[2]:
                    index1=s.rfind('/')
                    temp.append(s[index1+4:index1+7])
                    

                if i == 0:
                    query_c = outputs[0]
                    query_y = query_y_batch
                    query_pname=temp
                else:
                    query_c = T.cat([query_c, outputs[0]], 0)
                    query_y = T.cat([query_y, query_y_batch], 0)
                    query_pname=query_pname+temp
                pbar.update(1)

    mAP = Evaluate_mAP(C, args.N_books, gallery_c.type(T.int), query_c, gallery_y, query_y, device,gallery_pname,query_pname, args.Top_N)
    return mAP


In [6]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import *
from Retrieval import *
from dataset import SingleData
import math


def get_data_list(data_path, ratio=0.0):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    half=math.floor(len(eval_img_list)/2)
    print(half)
    eval_=eval_img_list[:half]
    test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list


def get_args_parser():
    parser = argparse.ArgumentParser('SPQ', add_help=False)

    parser.add_argument('--gpu_id', default="0", type=str, help="""Define GPU id.""")
    parser.add_argument('--if_download', default=False, type=bool, help="""Whether to download the dataset or not.""")
    parser.add_argument('--data_dir', default="./data", type=str, help="""Path of the dataset to be installed.""")
    # parser.add_argument('--batch_size', default=256, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--batch_size', default=32, type=int, help="""Training mini-batch size.""")
    parser.add_argument('--num_workers', default=4, type=int, help="""Number of data loading workers per GPU.""")
    parser.add_argument('--input_size', default=216, type=int, help="""Input image size, default is set to CIFAR10.""")

    parser.add_argument('--N_books', default=8, type=int, help="""The number of the codebooks.""")
    parser.add_argument('--N_words', default=16, type=int, help="""The number of the codewords. It should be a power of two.""")
    parser.add_argument('--L_word', default=16, type=int, help="""Dimensionality of the codeword.""")
    parser.add_argument('--soft_quantization_scale', default=5.0, type=float, help="""Soft-quantization scaling parameter.""")
    parser.add_argument('--contrastive_temperature', default=0.5, type=float, help="""Contrastive learning Temperature scaling parameter.""")
    
    parser.add_argument('--num_cls', default="3", type=int, help="""The number of classes in the dataset for evaluation, default is set to CIFAR10""")
    parser.add_argument('--eval_epoch', default=50, type=int, help="""Compute mAP for Every N-th epoch.""")
    # parser.add_argument('--eval_epoch', default=100, type=int, help="""Compute mAP for Every N-th epoch.""")
    parser.add_argument('--output_dir', default=".", type=str, help="""Path to save logs and checkpoints.""")
    parser.add_argument('--Top_N', default=100, type=int, help="""Top N number of images to be retrieved for evaluation.""")
    
    return parser
class CQCLoss(T.nn.Module):

    def __init__(self, device, batch_size, tau_cqc):
        super(CQCLoss, self).__init__()
        self.batch_size = batch_size
        self.tau_cqc = tau_cqc
        self.device = device
        self.COSSIM = T.nn.CosineSimilarity(dim=-1)
        self.CE = T.nn.CrossEntropyLoss(reduction="sum")
        self.get_corr_mask = self._get_correlated_mask().type(T.bool)

    def _get_correlated_mask(self):
        diag = np.eye(2 * self.batch_size)
        l1 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=-self.batch_size)
        l2 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=self.batch_size)
        mask = T.from_numpy((diag + l1 + l2))
        mask = (1 - mask).type(T.bool)
        return mask.to(self.device)

    def forward(self, Xa, Xb, Za, Zb):

        XaZb = T.cat([Xa, Zb], dim=0)
        XbZa = T.cat([Xb, Za], dim=0)

        Cossim_ab = self.COSSIM(XaZb.unsqueeze(1), XaZb.unsqueeze(0))
        Rab = T.diag(Cossim_ab, self.batch_size)
        Lab = T.diag(Cossim_ab, -self.batch_size)
        Pos_ab = T.cat([Rab, Lab]).view(2 * self.batch_size, 1)
        Neg_ab = Cossim_ab[self.get_corr_mask].view(2 * self.batch_size, -1)

        Cossim_ba = self.COSSIM(XbZa.unsqueeze(1), XbZa.unsqueeze(0))
        Rba = T.diag(Cossim_ba, self.batch_size)
        Lba = T.diag(Cossim_ba, -self.batch_size)    
        Pos_ba = T.cat([Rba, Lba]).view(2 * self.batch_size, 1)
        Neg_ba = Cossim_ba[self.get_corr_mask].view(2 * self.batch_size, -1)


        logits_ab = T.cat((Pos_ab, Neg_ab), dim=1)
        logits_ab /= self.tau_cqc

        logits_ba = T.cat((Pos_ba, Neg_ba), dim=1)
        logits_ba /= self.tau_cqc

        labels = T.zeros(2 * self.batch_size).to(self.device).long()
        
        loss = self.CE(logits_ab, labels) + self.CE(logits_ba, labels)
        return loss / (2 * self.batch_size)

def train_SPQ(args):
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
    device = T.device('cuda')
    # device = xm.xla_device()

    sz = args.input_size
    data_dir = args.data_dir
    batch_size = args.batch_size

    N_books = args.N_books
    N_words = args.N_words
    L_word = args.L_word
    tau_q = args.soft_quantization_scale
    tau_cqc = args.contrastive_temperature

    N_bits = int(N_books * np.sqrt(N_words))
    print('\033[91m' + '%d'%N_bits +  '-bit to retrieval' + '\033[0m')

    #Define the data augmentation following the setup of SimCLR
    Augmentation = nn.Sequential(
        Kg.RandomResizedCrop(size=(sz, sz)),
        Kg.RandomHorizontalFlip(p=0.5),
        Kg.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.8),
        Kg.RandomGrayscale(p=0.2),
        Kg.RandomGaussianBlur((int(0.1 * sz), int(0.1 * sz)), (0.1, 2.0), p=0.5))

    transform = transforms.ToTensor()
    
    class_name, train_img_list = get_data_list('/content/drive/MyDrive/dataSets/CRC/train/')
    train_data_set=SingleData(class_name, train_img_list, transform)
    
    # trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=args.if_download, transform=transform)
    trainloader = T.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # trainloader = T.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
    # testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
    class Quantization_Head(nn.Module):
        def __init__(self, N_words, N_books, L_word, tau_q):
            super(Quantization_Head, self).__init__()
            self.fc = nn.Linear(512, N_books * L_word)
            nn.init.xavier_uniform_(self.fc.weight)

            # Codebooks
            self.C = T.nn.Parameter(Variable((T.randn(N_words, N_books * L_word)).type(T.float32), requires_grad=True))
            nn.init.xavier_uniform_(self.C)

            self.N_books = N_books
            self.L_word = L_word
            self.tau_q = tau_q

        def forward(self, input):
            X = self.fc(input)
            Z = Soft_Quantization(X, self.C, self.N_books, self.tau_q)
            return X, Z
        
    Q = Quantization_Head(N_words, N_books, L_word, tau_q)
    net = nn.Sequential(ResNet_Baseline(BasicBlock, [2, 2, 2, 2]), Q)
    net.load_state_dict(T.load('/content/drive/MyDrive/models/SPQ/349_32_0.1871_checkpoint.pth'))
    net.cuda(device)
    # mx    = xmp.MpModelWrapper(net)
    # net  = mx.to(device)

    criterion = CQCLoss(device, batch_size, tau_cqc)

    optimizer = T.optim.Adam(net.parameters(), lr=3e-4, weight_decay=10e-6)
    scheduler = CosineAnnealingLR(optimizer, T_max=len(trainloader), eta_min=0, last_epoch=-1)

    MAX_mAP = 0.0
    mAP = 0.0

    for epoch in range(350,600):  # loop over the dataset multiple times

        print('Epoch: %d, Learning rate: %.4f' % (epoch, scheduler.get_last_lr()[0]))
        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs = data[0].to(device)
            Ia = Augmentation(inputs)
            Ib = Augmentation(inputs)

            optimizer.zero_grad()

            Xa, Za = net(Ia)
           
            return 1;
        #     Xb, Zb = net(Ib)

        #     loss = criterion(Xa, Xb, Za, Zb)
        #     loss.backward()
        #     optimizer.step()

        #     running_loss += loss.item()

        #     if (i+1) % 10 == 0:    # print every 10 mini-batches
        #         print('[%3d] loss: %.4f, mAP: %.4f, MAX mAP: %.4f' %
        #             (i+1, running_loss / 10, mAP, MAX_mAP))
        #         running_loss = 0.0

        # if epoch >= 10:
        #     scheduler.step()
        
        # if (epoch+1) % args.eval_epoch == 0:
        #     mAP = DoRetrieval(device, args, net, Q.C)
        #     if mAP > MAX_mAP:
        #         Result_path = os.path.join(args.output_dir, "%d_%d_%.4f_checkpoint.pth"%(epoch,N_bits, mAP))
        #         T.save(net.state_dict(), Result_path)
        #         MAX_mAP = mAP

if __name__ == '__main__':
    parser = argparse.ArgumentParser('SPQ', parents=[get_args_parser()])
    args = parser.parse_args("")
    train_SPQ(args)


32-bit to retrieval
0
['s1', 's2', 's3']
Epoch: 350, Learning rate: 0.0003
torch.Size([32, 512, 27, 27])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512])
torch.Size([32, 512])


In [ ]:
model.load_state_dict(torch.load(weight_path))

NameError: name 'mAP' is not defined

In [ ]:
print(args)

Namespace(gpu_id='0', if_download=False, data_dir='./data', batch_size=32, num_workers=8, input_size=96, N_books=8, N_words=16, L_word=16, soft_quantization_scale=5.0, contrastive_temperature=0.5, num_cls=3, eval_epoch=50, output_dir='.', Top_N=8)


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
device = T.device('cuda')

sz = args.input_size
data_dir = args.data_dir
batch_size = args.batch_size

N_books = args.N_books
N_words = args.N_words
L_word = args.L_word
tau_q = args.soft_quantization_scale
tau_cqc = args.contrastive_temperature

N_bits = int(N_books * np.sqrt(N_words))
print('\033[91m' + '%d'%N_bits +  '-bit to retrieval' + '\033[0m')

#Define the data augmentation following the setup of SimCLR
Augmentation = nn.Sequential(
    Kg.RandomResizedCrop(size=(sz, sz)),
    Kg.RandomHorizontalFlip(p=0.5),
    Kg.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.8),
    Kg.RandomGrayscale(p=0.2),
    Kg.RandomGaussianBlur((int(0.1 * sz), int(0.1 * sz)), (0.1, 2.0), p=0.5))

transform = transforms.ToTensor()

class_name, train_img_list = get_data_list('D:\\original_images_5\\test-patches\\trainset\\SPQ\\final96\\train\\final')
train_data_set=SingleData(class_name, train_img_list, transform)

trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=args.if_download, transform=transform)
trainloader = T.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
# trainloader = T.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=args.num_workers)
testset = torchvision.datasets.CIFAR10(root=args.data_dir, train=False, download=args.if_download, transform=transforms.ToTensor())
class Quantization_Head(nn.Module):
    def __init__(self, N_words, N_books, L_word, tau_q):
        super(Quantization_Head, self).__init__()
        self.fc = nn.Linear(512, N_books * L_word)
        nn.init.xavier_uniform_(self.fc.weight)

        # Codebooks
        self.C = T.nn.Parameter(Variable((T.randn(N_words, N_books * L_word)).type(T.float32), requires_grad=True))
        nn.init.xavier_uniform_(self.C)

        self.N_books = N_books
        self.L_word = L_word
        self.tau_q = tau_q

    def forward(self, input):
        X = self.fc(input)
        Z = Soft_Quantization(X, self.C, self.N_books, self.tau_q)
        return X, Z

Q = Quantization_Head(N_words, N_books, L_word, tau_q)
net = nn.Sequential(ResNet_Baseline(BasicBlock, [2, 2, 2, 2]), Q)
net.load_state_dict(T.load('D:\\models\\SPQ-main\\32_0.6476_checkpoint.pth'))
net.cuda(device)

criterion = CQCLoss(device, batch_size, tau_cqc)

optimizer = T.optim.Adam(net.parameters(), lr=3e-4, weight_decay=10e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=len(trainloader), eta_min=0, last_epoch=-1)

MAX_mAP = 0.0
mAP = 0.0

32-bit to retrieval
0
['s1', 's2', 's3']


In [ ]:
mAP = DoRetrieval(device, args, net, Q.C)

Do Retrieval!
7500
['s1', 's2', 's3']
15
['s1', 's2', 's3']


Evaluate mAP    94%|█████████▍| 2805/2970 [02:00<00:07, 23.19it/s]


In [ ]:
mAP

tensor(0.6476, device='cuda:0')